In [28]:
# BigQuery Setup
# Importing Libraries and Credentials
import pandas as pd
import numpy as np
import seaborn as sns
from google.cloud import bigquery
from google.oauth2 import service_account
# ignore warnings
from warnings import filterwarnings
filterwarnings("ignore")


%load_ext google.cloud.bigquery

credentials = service_account.Credentials.from_service_account_file('/Users/ssamilozkan/Desktop/BigQuery/config.json')

project_id = 'dbt-bigquery-setup-369911'
client = bigquery.Client(credentials= credentials, project=project_id)

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### Identify duplicate rows
Seeing a sample amount of data may give you greater intuition for what is included in the dataset. To preview sample rows from the table without using SQL, click the Preview tab.

Scan and scroll through the rows. There is no singular field that uniquely identifies a row, so you need advanced logic to identify duplicate rows.

Your query uses the SQL GROUP BY function on every field and counts (COUNT) where there are rows that have the same values across every field.

- If every field is unique, the COUNT will return 1 as there are no other groupings of rows with the exact same value for all fields.

- If there is a row with the same values for all fields, they will be grouped together and the COUNT will be greater than 1. The last part of the query is an aggregation filter using HAVING to only show the results that have a COUNT of duplicates greater than 1.

- Copy and paste the following query into the query EDITOR, then RUN to find duplicate records across all columns field. If the EDITOR tab isn't visible, then click COMPOSE NEW QUERY.

In [42]:
%%bigquery
SELECT COUNT(*) as num_duplicate_rows, * FROM
`data-to-insights.ecommerce.all_sessions_raw`
GROUP BY
fullVisitorId, channelGrouping, time, country, city, totalTransactionRevenue, transactions, timeOnSite, pageviews, sessionQualityDim, date, visitId, type, productRefundAmount, productQuantity, productPrice, productRevenue, productSKU, v2ProductName, v2ProductCategory, productVariant, currencyCode, itemQuantity, itemRevenue, transactionRevenue, transactionId, pageTitle, searchKeyword, pagePathLevel1, eCommerceAction_type, eCommerceAction_step, eCommerceAction_option
HAVING num_duplicate_rows > 1;

,num_duplicate_rows,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,...,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,2,315410525784859471,Organic Search,253390,Ecuador,not available in demo dataset,<NA>,<NA>,292,24,...,<NA>,<NA>,<NA>,None,Men's Outerwear | Apparel | Google Merchandise...,None,/google+redesign/,2,1,None
1,2,552850508976566436,Organic Search,67845,Italy,not available in demo dataset,<NA>,<NA>,97,23,...,<NA>,<NA>,<NA>,None,Headgear | Apparel | Google Merchandise Store,None,/store.html,1,1,None
2,2,0409094433310347683,Social,144021,Mexico,not available in demo dataset,<NA>,<NA>,175,36,...,<NA>,<NA>,<NA>,None,Apparel | Google Merchandise Store,None,/google+redesign/,2,1,None
3,2,8236220456907982969,Organic Search,62222,United States,not available in demo dataset,<NA>,<NA>,822,14,...,<NA>,<NA>,<NA>,None,Apparel | Google Merchandise Store,None,/google+redesign/,1,1,None
4,2,3438386296748685726,Referral,3121610,United States,New York,204500000,1,4941,45,...,<NA>,<NA>,<NA>,ORD201610242248,Checkout Confirmation,None,/ordercompleted.html,6,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,22,4884778281436388667,Referral,441425,United States,Cambridge,555000000,1,485,75,...,<NA>,<NA>,<NA>,None,Checkout Review,None,/revieworder.html,5,3,Review
611,2,7890468841809778203,Referral,403943,United States,San Jose,127000000,1,453,30,...,<NA>,<NA>,<NA>,ORD201612031681,Checkout Confirmation,None,/ordercompleted.html,6,1,None
612,3,1915538933685278364,Referral,580726,United States,San Jose,<NA>,<NA>,638,22,...,<NA>,<NA>,<NA>,None,Checkout Your Information,None,/yourinfo.html,5,1,Billing and Shipping
613,2,5186802869072248264,Direct,624844,United States,Mountain View,199580000,1,1009,62,...,<NA>,<NA>,<NA>,None,Checkout Your Information,None,/yourinfo.html,5,1,Billing and Shipping


### Analyze the new all_sessions table

In this section you use a deduplicated table called all_sessions.

**Scenario:** Your data analyst team has provided you with this query, and your schema experts have identified the key fields that must be unique for each record per your schema.

- Run the query to confirm that no duplicates exist, this time in the all_sessions table:

In [43]:
%%bigquery
#standardSQL
# schema: https://support.google.com/analytics/answer/3437719?hl=en
SELECT
fullVisitorId, # the unique visitor ID
visitId, # a visitor can have multiple visits
date, # session date stored as string YYYYMMDD
time, # time of the individual site hit  (can be 0 to many per visitor session)
v2ProductName, # not unique since a product can have variants like Color
productSKU, # unique for each product
type, # a visitor can visit Pages and/or can trigger Events (even at the same time)
eCommerceAction_type, # maps to ‘add to cart', ‘completed checkout'
eCommerceAction_step,
eCommerceAction_option,
  transactionRevenue, # revenue of the order
  transactionId, # unique identifier for revenue bearing transaction
COUNT(*) as row_count
FROM
`data-to-insights.ecommerce.all_sessions`
GROUP BY 1,2,3 ,4, 5, 6, 7, 8, 9, 10,11,12
HAVING row_count > 1 # find duplicates

,fullVisitorId,visitId,date,time,v2ProductName,productSKU,type,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option,transactionRevenue,transactionId,row_count


### Write basic SQL on ecommerce data

In this section, you query for insights on the ecommerce dataset.

Write a query that shows total unique visitors
Your query determines the total views by counting product_views and the number of unique visitors by counting `fullVisitorID`.

Write this query in the editor:

In [44]:
%%bigquery
#standardSQL
SELECT
  COUNT(*) AS product_views,
  COUNT(DISTINCT fullVisitorId) AS unique_visitors
FROM `data-to-insights.ecommerce.all_sessions`;

,product_views,unique_visitors
0,21493109,389934


Now write a query that shows total unique visitors(fullVisitorID) by the referring site (channelGrouping):

In [29]:
%%bigquery
#standardSQL
SELECT
  COUNT(DISTINCT fullVisitorId) AS unique_visitors,
  channelGrouping
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY channelGrouping
ORDER BY channelGrouping DESC;

,unique_visitors,channelGrouping
0,38101,Social
1,57308,Referral
2,11865,Paid Search
3,211993,Organic Search
4,3067,Display
5,75688,Direct
6,5966,Affiliates
7,62,(Other)


Write a query to list all the unique product names (`v2ProductName`) alphabetically:

- This query returns a total of 633 products (rows).

In [30]:
%%bigquery
#standardSQL
SELECT
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY ProductName
ORDER BY ProductName

,ProductName
0,1 oz Hand Sanitizer
1,14oz Ceramic Google Mug
2,15 oz Ceramic Mug
3,"15"" Android Squishable - Online"
4,16 oz. Hot and Cold Tumbler
...,...
628,YouTube Women's Short Sleeve Tri-blend Badge T...
629,YouTube Women's Short Sleeve Tri-blend Badge T...
630,YouTube Womens 3/4 Sleeve Baseball Raglan Whit...
631,YouTube Wool Heather Cap Heather/Black


Write a query to list the five products with the most views (`product_views`) from all visitors (include people who have viewed the same product more than once). Your query counts number of times a product (`v2ProductName`) was viewed (`product_views`), puts the list in descending order, and lists the top 5 entries:


In [31]:
%%bigquery 
#standardSQL
SELECT
  COUNT(*) AS product_views,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,ProductName
0,316482,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,22 oz YouTube Bottle Infuser
2,210700,YouTube Men's Short Sleeve Hero Tee Black
3,202205,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,YouTube Custom Decals


Refine the query to no longer double-count product views for visitors who have viewed a product many times. Each distinct product view should only count once per visitor.

In [32]:
%%bigquery
WITH unique_product_views_by_person AS (
##find each unique product viewed by each visitor
SELECT
 fullVisitorId,
 (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY fullVisitorId, v2ProductName )
##aggregate the top viewed products and sort them
SELECT
  COUNT(*) AS unique_view_count,
  ProductName
FROM unique_product_views_by_person
GROUP BY ProductName
ORDER BY unique_view_count DESC
LIMIT 5

,unique_view_count,ProductName
0,152358,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,143770,22 oz YouTube Bottle Infuser
2,127904,YouTube Men's Short Sleeve Hero Tee Black
3,122051,YouTube Twill Cap
4,121288,YouTube Custom Decals


Next, expand your previous query to include the total number of distinct products ordered and the total number of total units ordered (productQuantity):

In [33]:
%%bigquery
#standardSQL
SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,v2ProductName
0,316482,3158,6352,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,22 oz YouTube Bottle Infuser
2,210700,949,1114,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,YouTube Custom Decals


Expand the query to include the average amount of product per order (total number of units ordered/total number of orders, or SUM(productQuantity)/COUNT(productQuantity)):

In [36]:
%%bigquery
#standardSQL
SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS orders,
  SUM(productQuantity) AS quantity_product_ordered,
  SUM(productQuantity) / COUNT(productQuantity) AS avg_per_order,
  (v2ProductName) AS ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE type = 'PAGE'
GROUP BY v2ProductName
ORDER BY product_views DESC
LIMIT 5;

,product_views,orders,quantity_product_ordered,avg_per_order,ProductName
0,316482,3158,6352,2.011400,Google Men's 100% Cotton Short Sleeve Hero Tee...
1,221558,508,4769,9.387795,22 oz YouTube Bottle Infuser
2,210700,949,1114,1.173867,YouTube Men's Short Sleeve Hero Tee Black
3,202205,2713,8072,2.975304,Google Men's 100% Cotton Short Sleeve Hero Tee...
4,200789,1703,11336,6.656489,YouTube Custom Decals


## Challenge 1: Calculate a conversion rate

- Write a conversion rate query for products with these qualities:
    - More than 1000 units were added to a cart or ordered

    - AND are not frisbees

- Answer these questions:
    - How many distinct times was the product part of an order (either complete or incomplete order)?

    - How many total units of the product were part of orders (either complete or incomplete)?

    - Which product had the highest conversion rate?

Complete the following partial query:

In [37]:
%%bigquery
#standardSQL
SELECT
  COUNT(*) AS product_views,
  COUNT(productQuantity) AS potential_orders,
  SUM(productQuantity) AS quantity_product_added,
  (COUNT(productQuantity) / COUNT(*)) AS conversion_rate,
  v2ProductName
FROM `data-to-insights.ecommerce.all_sessions`
WHERE LOWER(v2ProductName) NOT LIKE '%frisbee%'
GROUP BY v2ProductName
HAVING quantity_product_added > 1000
ORDER BY conversion_rate DESC
LIMIT 10;

,product_views,potential_orders,quantity_product_added,conversion_rate,v2ProductName
0,683,240,1428,0.351391,Google 25 oz Clear Stainless Steel Bottle
1,3667,1282,1622,0.349605,Google Men's Bike Short Sleeve Tee Charcoal
2,629,194,1101,0.308426,Android Men's Paradise Short Sleeve Tee Olive
3,6897,1524,1856,0.220966,BLM Sweatshirt
4,147729,22993,140734,0.155643,Nest® Learning Thermostat 3rd Gen-USA - Stainl...
5,1574,207,1284,0.131512,Google Leather Journal-Black
6,1498,195,1406,0.130174,Google Leather Journal-Brown
7,154713,19357,30201,0.125116,Nest® Cam Outdoor Security Camera - USA
8,20584,2461,41975,0.119559,Recycled Paper Journal Set
9,153091,17980,10000025529,0.117446,Nest® Cam Indoor Security Camera - USA


## Challenge 2: Track visitor checkout progress

- Write a query that shows the eCommerceAction_type and the distinct count of fullVisitorId associated with each type.

- You are given this mapping for the action type: 
    - Unknown = 0 
    - Click through of product lists = 1 
    - Product detail views = 2
    - Add product(s) to cart = 3 
    - Remove product(s) from cart = 4 
    - Check out = 5 
    - Completed purchase = 6 
    - Refund of purchase = 7 
    - Checkout options = 8

Use a Case Statement to add a new column to your previous query to display the eCommerceAction_type label (such as “Completed purchase”).

In [39]:
%%bigquery
#standardSQL
SELECT
  COUNT(DISTINCT fullVisitorId) AS number_of_unique_visitors,
  eCommerceAction_type,
  CASE eCommerceAction_type
  WHEN '0' THEN 'Unknown'
  WHEN '1' THEN 'Click through of product lists'
  WHEN '2' THEN 'Product detail views'
  WHEN '3' THEN 'Add product(s) to cart'
  WHEN '4' THEN 'Remove product(s) from cart'
  WHEN '5' THEN 'Check out'
  WHEN '6' THEN 'Completed purchase'
  WHEN '7' THEN 'Refund of purchase'
  WHEN '8' THEN 'Checkout options'
  ELSE 'ERROR'
  END AS eCommerceAction_type_label
FROM `data-to-insights.ecommerce.all_sessions`
GROUP BY eCommerceAction_type
ORDER BY eCommerceAction_type;

,number_of_unique_visitors,eCommerceAction_type,eCommerceAction_type_label
0,389240,0,Unknown
1,122728,1,Click through of product lists
2,122477,2,Product detail views
3,56010,3,Add product(s) to cart
4,12015,4,Remove product(s) from cart
5,30408,5,Check out
6,19988,6,Completed purchase


## Challenge 3: Track abandoned carts from high quality sessions

- Write a query using aggregation functions that returns the unique session IDs of those visitors who have added a product to their cart but never completed checkout (abandoned their shopping cart).

In [40]:
%%bigquery
#standardSQL
# high quality abandoned carts
SELECT  
  #unique_session_id
  CONCAT(fullVisitorId,CAST(visitId AS STRING)) AS unique_session_id,
  sessionQualityDim,
  SUM(productRevenue) AS transaction_revenue,
  MAX(eCommerceAction_type) AS checkout_progress
FROM `data-to-insights.ecommerce.all_sessions`
WHERE sessionQualityDim > 60 # high quality session
GROUP BY unique_session_id, sessionQualityDim
HAVING
  checkout_progress = '3' # 3 = added to cart
  AND (transaction_revenue = 0 OR transaction_revenue IS NULL)

,unique_session_id,sessionQualityDim,transaction_revenue,checkout_progress
0,16616911788905976891501125270,71,<NA>,3
1,07545176369523311931500935220,93,<NA>,3
2,60526958981638682581501026186,63,<NA>,3
3,17113683097965803971501354680,81,<NA>,3
4,90557560612607972081501076878,80,<NA>,3
...,...,...,...,...
421,74066146785746721421501421700,75,<NA>,3
422,92279105513493566651499983954,91,<NA>,3
423,1992335154444956761499805528,89,<NA>,3
424,98252349784880986991500344716,78,<NA>,3
